# Working with Time Zones

## Python's Time Zone Model

The `tzinfo` class is an abstract base class, and you are required to implement three methods:

```python
class tzinfo:
    def tzname(self, dt):
        raise NotImplementedError()

    def utcoffset(self, dt):
        raise NotImplementedError()
        
    def dst(self, dt):
        raise NotImplementedError()
        
    ...
```

- `tzname`: Return the name or abbreviation for the time zone at the given datetime.
- `utcoffset`: Return a `datetime.timedelta` representing the offset from UTC at the given datetime.
- `dst`: Return a `datetime.timedelta` representing the amount of the daylight saving time offset at a given time. *This method is rarely useful*.

The reason these are *methods* rather than fixed values is that a `tzinfo` represents a mapping between a datetime and the set of rules that applies at that time in this zone. The same concrete `tzinfo` class is used for many `datetime`s.

### Example: Eastern Time

In [1]:
from dateutil import tz
from datetime import datetime, timedelta

In [2]:
def print_tzinfo(dt):
    """Convenience function for printing the time zone information of a datetime"""
    print("dt.tzname: " + str(dt.tzname()))
    print("dt.utcoffset: " + str(dt.utcoffset() / timedelta(hours=1)) + " hours")
    print("dt.dst: " + str(dt.dst() / timedelta(hours=1)) + " hours")

In [3]:
EASTERN = tz.gettz("America/New_York")

In [4]:
dt_std = datetime(2019, 3, 1, 12, tzinfo=EASTERN)

In [5]:
print_tzinfo(dt_std)

dt.tzname: EST
dt.utcoffset: -5.0 hours
dt.dst: 0.0 hours


In [6]:
dt_dst = datetime(2019, 4, 1, 12, tzinfo=EASTERN)

In [7]:
print_tzinfo(dt_dst)

dt.tzname: EDT
dt.utcoffset: -4.0 hours
dt.dst: 1.0 hours


### Exercise: Implement a UTC time zone class

The simplest `tzinfo` classes to implement are fixed offsets from UTC and UTC itself, which have offsets that do not change as a function of `datetime`. To practice a basic tzinfo implementation, implement a class `UTC` representing the UTC time zone.

In [8]:
from datetime import tzinfo

In [9]:
from datetime import timezone

In [10]:
datetime(2019, 1, 1, tzinfo=timezone.utc).dst()

In [11]:
class UTC(tzinfo):
    def tzname(self, dt):
        return "UTC"
    
    def utcoffset(self, dt):
        return timedelta(0)
    
    def dst(self, dt):
        return None

In [12]:
# Tests
test_cases = [
    datetime(1, 1, 1),
    datetime(1857, 2, 7),
    datetime(1970, 1, 1),
    datetime(2010, 3, 21, 2, 16)
]

utc = UTC()
for dt in test_cases:
    dt_act = dt.replace(tzinfo=utc)
    dt_exp = dt.replace(tzinfo=timezone.utc)
    
    assert dt_act.tzname() == dt_exp.tzname(), f'tznames must match for {dt}'
    assert dt_act.utcoffset() == dt_exp.utcoffset(), f'utcoffset must match for {dt}'
    assert dt_act.dst() == dt_exp.dst(), f'dst must match for {dt}'

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [13]:
from helper_functions import print_dt_tzinfo

In [14]:
dt = datetime(2017, 8, 11, 14, tzinfo=tz.gettz('America/New_York'))
print_dt_tzinfo(dt)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you have a naive wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [15]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

2017-08-11 14:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [16]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


### `pytz`

In `pytz`, `datetime.astimezone()` still works as expected:

In [17]:
import pytz

In [18]:
print_dt_tzinfo(dt.astimezone(pytz.timezone('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


But the constructor and `.replace` methods fail horribly:

In [19]:
print_dt_tzinfo(dt.replace(tzinfo=pytz.timezone('America/Los_Angeles')))

2017-08-11 14:00:00-0753
    tzname:   LMT;      UTC Offset:  -7.88h;        DST:      0.0h


This is because `pytz` uses a different time zone model. `pytz`'s time zone model implements the `tzinfo` interface *statically*, which is to say that the `tzname`, `utcoffset` and `dst` are all calculated eagerly, when the `tzinfo` is attached to the `datetime`.

In order to accomplish this, `pytz` expects all `tzinfo` objects to be attached to the `datetime` *by the time zone object itself*, using the `localize()` method:

In [20]:
EASTERN_p = pytz.timezone('America/New_York')
dt_p = EASTERN_p.localize(datetime(2017, 8, 11, 14))
print_dt_tzinfo(dt_p)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


This also means that unlike with normal `tzinfo` objects, after you've done some arithmetic on a `pytz`-aware `datetime` object, you must `normalize` it:

In [21]:
print('dateutil:')
print_dt_tzinfo(dt + timedelta(days=180))
print('')
print('pytz')
print_dt_tzinfo(dt_p + timedelta(days=180))

dateutil:
2018-02-07 14:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

pytz
2018-02-07 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


In [22]:
print_dt_tzinfo(EASTERN_p.normalize(dt_p + timedelta(days=180)))

2018-02-07 13:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


### `datetime.now` and `datetime.fromtimestamp`

The alternate constructors `now()` and `fromtimestamp` both take an optional argument `tz`:


```python
    def now(tz=None):
        """Get the datetime representing the current time"""
        # ...

    def fromtimestamp(self, timestamp, tz=None):
        """ Return the datetime corresponding to a POSIX timestamp """
        # ...
```

If the `tz` parameter is not passed, they will return a *naïve* datetime, representing the time in your system local time.

In [23]:
datetime.fromtimestamp(1577854800.0)

datetime.datetime(2020, 1, 1, 0, 0)

If you want an *aware* timezone from a timestamp (or the current time), pass it to the `tz` parameter, and it will calculate the correct `datetime` matching that time zone. This works with both `pytz` and `dateutil.tz`:

In [24]:
print("pytz:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN_p))
print("")
print("dateutil:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN))

pytz:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

dateutil:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: There are also the semi-deprecated `datetime.utcnow()` and `datetime.utcfromtimestamp()` functions. These  return a *naïve* datetime expressed in UTC. It is almost always better to simply pass `tz=UTC` (where `UTC` is some time zone object), which will give you an *aware* `datetime`.

### Exercise: Current time in multiple time zones


To practice using time zones, try writing a function that takes as inputs a list of IANA time zones and prints the time in each time zone.

So, for example:

```python
>>> now_in_zones(['Asia/Tokyo',
...               'Europe/Berlin',
...               'America/New_York',
...               'America/Los_Angeles'])
```
```
Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
```

In [25]:
def now_in_zones(tz_list):
    # For now I'll fib and pretend the current datetime is 2020
    dt_utc = datetime(2020, 1, 1, tzinfo=tz.gettz('America/New_York'))
    for tzstr in tz_list:
        dt = dt_utc.astimezone(tz.gettz(tzstr))
        print(f"{tzstr + ':':<25} {dt}")
        
now_in_zones(['Asia/Tokyo',
              'Europe/Berlin',
              'America/New_York',
              'America/Los_Angeles'])
    

Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00


## Ambiguous and imaginary times

### Ambiguous times
An *ambiguous* time is when the same "wall time" occurs more than once, such as during a DST to STD transition.

In [26]:
NYC = tz.gettz('America/New_York')

In [27]:
dt1 = datetime(2004, 10, 31, 4, 30, tzinfo=tz.UTC)
for i in range(4):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    ambig_str = 'Ambiguous' if tz.datetime_ambiguous(dt) else 'Unambiguous'
    print(f'{dt} | {dt.tzname()} |  {ambig_str}')

2004-10-31 00:30:00-04:00 | EDT |  Unambiguous
2004-10-31 01:30:00-04:00 | EDT |  Ambiguous
2004-10-31 01:30:00-05:00 | EST |  Ambiguous
2004-10-31 02:30:00-05:00 | EST |  Unambiguous


### [PEP 495: Local Time Disambiguation](https://www.python.org/dev/peps/pep-0495/)
- First introduced in Python 3.6
- Introduces the `fold` attribute of `datetime`
- Changes aware datetime comparison around ambiguous times

PEP 495 makes whether you are on the fold side a *property of the `datetime`*:

In [28]:
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))
print('')
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=NYC))

2004-10-31 01:30:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h

2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: `fold=1` represents the *second* instance of an ambiguous `datetime`.

### Imaginary times
*Imaginary* times are wall times that never occur in a given time zone, such as during an STD to DST transition:

In [29]:
dt1 = datetime(2004, 4, 4, 6, 30, tzinfo=tz.UTC)
for i in range(3):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    print(f'{dt} | {dt.tzname()} ')

2004-04-04 01:30:00-05:00 | EST 
2004-04-04 03:30:00-04:00 | EDT 
2004-04-04 04:30:00-04:00 | EDT 


### Handling ambiguous times

Both `dateutil` and `pytz` will automatically give you the right *absolute time* if converting from an absolute time.

In [30]:
dt1 = datetime(2004, 10, 31, 6, 30, tzinfo=tz.UTC)   # This is in the fold in EST

In [31]:
# dateutil
dt_du = dt1.astimezone(tz.gettz('America/New_York'))
print(repr(dt_du))
print_dt_tzinfo(dt_du)

datetime.datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=tzfile('/usr/share/zoneinfo/America/New_York'))
2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


In [32]:
# pytz
dt_pytz = dt1.astimezone(pytz.timezone('America/New_York'))
print(repr(dt_pytz))    # Note that pytz doesn't set the fold attribute
print_dt_tzinfo(dt_pytz)

datetime.datetime(2004, 10, 31, 1, 30, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>)
2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


For backwards compatibility, `dateutil` provides a `tz.enfold` method to add a `fold` attribute if necessary:

In [33]:
dt = datetime(2004, 10, 31, 1, 30, tzinfo=NYC)
tz.enfold(dt)

datetime.datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=tzfile('/usr/share/zoneinfo/America/New_York'))

```python
Python 2.7.12
Type "help", "copyright", "credits" or "license" for more information.
>>> from datetime import datetime
>>> from dateutil import tz
>>> dt = datetime(2004, 10, 31, 1, 30, tzinfo=tz.gettz('US/Eastern'))
>>> tz.enfold(dt)
_DatetimeWithFold(2004, 10, 31, 1, 30, tzinfo=tzfile('/usr/share/zoneinfo/US/Eastern'))
>>> tz.enfold(dt).tzname()
'EST'
>>> dt.tzname()
'EDT'
```

To detect ambiguous times, `dateutil` provides `tz.datetime_ambiguous`:

In [34]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))

True

In [35]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30), NYC)

True

In [36]:
dt_0 = datetime(2004, 10, 31, 0, 30, tzinfo=NYC)
for i in range(3):
    dt_i = dt_0 + timedelta(hours=i)
    dt_i = tz.enfold(dt_i, tz.datetime_ambiguous(dt_i))
    print(f'{dt_i} (fold={dt_i.fold})')

2004-10-31 00:30:00-04:00 (fold=0)
2004-10-31 01:30:00-05:00 (fold=1)
2004-10-31 02:30:00-05:00 (fold=0)


**Note:** `fold` is ignored when `datetime` is not ambiguous:

In [37]:
for i in range(3):
    dt_i = tz.enfold(dt_0 + timedelta(hours=i), fold=1)
    print(f'{dt_i} (fold={dt_i.fold})')

2004-10-31 00:30:00-04:00 (fold=1)
2004-10-31 01:30:00-05:00 (fold=1)
2004-10-31 02:30:00-05:00 (fold=1)


### Handling imaginary times

While functions that convert from an *absolute time* to another absolute time (e.g. `astimezone`, `fromtimestamp` and `now`) will never create an imaginary time, it is possible to create imaginary times with functions that manipulate the naive portion of the date, such as the constructor, arithmetic and `replace`.

In order to determine if you have created an imaginary time after one of these operations, you can use `dateutil`'s `tz.datetime_exists()`:

In [38]:
dt_0 = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
for i in range(3):
    dt = dt_0 + timedelta(hours=i)
    print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

2004-04-04 01:30:00-05:00 (Exists)
2004-04-04 02:30:00-04:00 (Imaginary)
2004-04-04 03:30:00-04:00 (Exists)


For the most part, if you are creating imaginary times, you want to "skip forward", to what the time *would be* if the transition had not happened: for example, if the datetime you have created is the result of an addition operation. For this, `dateutil` provides the `tz.resolve_imaginary` function:

In [39]:
dt = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
dt_imag = dt + timedelta(hours=1)   # 2004-04-04 02:30 is imaginary
print(f"Imaginary: {dt_imag}")
print(f"Resolved:  {tz.resolve_imaginary(dt_imag)}")

Imaginary: 2004-04-04 02:30:00-04:00
Resolved:  2004-04-04 03:30:00-04:00


This works for imaginary times other than 1 hour as well:

In [40]:
dt = datetime(1994, 12, 31, 9, tzinfo=tz.gettz('Pacific/Kiritimati'))
print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

1994-12-31 09:00:00-10:00 (Imaginary)


In [41]:
tz.resolve_imaginary(dt)

datetime.datetime(1995, 1, 1, 9, 0, tzinfo=tzfile('/usr/share/zoneinfo/Pacific/Kiritimati'))

All of these functions for handling ambiguous and imaginary times will work for `pytz` time zones as well, though `pytz` has its own way of handling this.

### `pytz`'s approach to handling ambiguous and imaginary times
#### Ambiguous times

`pytz` predates PEP 495, and in many ways its design was intended to solve the ambiguous and imaginary time problem before there was support for doing so in the standard library. Much of its non-standard interface is a consequence of the fact that without eagerly calculating time zone offsets, there is no way to specify which ambiguous wall time your `datetime` represents.

When localizing an ambiguous `datetime`, `pytz` will default to the *second* occurrence (i.e. the `fold=1` state, usually standard time):

In [42]:
NYC_pytz = pytz.timezone('America/New_York')
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30))
print_dt_tzinfo(dt_pytz)

2004-10-31 01:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


To get the *first* occurrence of a given wall time (i.e. the `fold=0` state, usually daylight saving time), pass `is_dst=True` to the `localize` function:

In [43]:
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30), is_dst=True)
print_dt_tzinfo(dt_pytz)

2004-10-31 01:30:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you want to *detect* ambiguous times, pass `is_dst=None`, and `pytz` will raise an `AmbiguousTimeError` if a datetime is ambiguous:

In [44]:
for hour in (0, 1):
    dt = datetime(2004, 10, 31, hour, 30)
    try:
        NYC_pytz.localize(dt, is_dst=None)
        print(f'{dt} | Unambiguous')
    except pytz.AmbiguousTimeError:
        print(f'{dt} | Ambiguous')

2004-10-31 00:30:00 | Unambiguous
2004-10-31 01:30:00 | Ambiguous


#### Imaginary times
When using `localize` on an imaginary `datetime`, `pytz` will create an imaginary time and assign it an offset based on `is_dst`:

In [45]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=True))
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=False))

2004-04-04 02:30:00-04:00
2004-04-04 02:30:00-05:00


The default for `is_dst` is `False`:

In [46]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30)))

2004-04-04 02:30:00-05:00


Again, setting `is_dst=None` will cause `pytz` to throw an error, this time `NonExistentTimeError`:

In [47]:
dt_0 = datetime(2004, 4, 4, 1, 30)
for i in range(3):
    try:
        dt = NYC_pytz.localize(dt_0 + timedelta(hours=i), is_dst=None)
        exist_str = 'Exists'
    except pytz.NonExistentTimeError:
        exist_str = 'Imaginary'

    print(f'{dt} ({exist_str})')

2004-04-04 01:30:00-05:00 (Exists)
2004-04-04 01:30:00-05:00 (Imaginary)
2004-04-04 03:30:00-04:00 (Exists)


### Exercise: Build a `pytz`-style exception-based localizer with `dateutil`

While `dateutil`'s interface for handling imaginary and ambiguous times is compatible with `pytz` zones, there is no built-in mechanism to throw *exceptions* when imaginary or ambiguous times are created. As an exercise, try to build a `localize` function that throws an exception using `dateutil.tz`'s functions for handling ambiguous and imaginary datetimes.

In [48]:
class AmbiguousTimeError(Exception):
    """Raised if an ambiguous time is detected"""

class NonExistentTimeError(Exception):
    """Raised if an imaginary time is detected"""
    
def localize(dt, tzi, is_dst=False):
    """
    Mimicks `pytz`'s `localize` function using the `fold` attribute.
    """
    if dt.tzinfo is not None:
        raise ValueError('localize can only be used with naive datetimes')

    if is_dst is None:
        # If is_dst is None, we want to raise an error for uncertain situations
        dt_out = dt.replace(tzinfo=tzi)
        if tz.datetime_ambiguous(dt_out):
            raise AmbiguousTimeError(f"Ambiguous time {dt} in zone {tzi}")
        elif not tz.datetime_exists(dt_out):
            raise NonExistentTimeError(f"Time {dt} does not exist in zone {tzi}")
    else:
        dt_out = dt.replace(fold=(not is_dst), tzinfo=tzi)

    return dt_out

In [49]:
# Tests
from contextlib import contextmanager

@contextmanager
def assert_raises(err_type):
    try:
        yield
    except err_type:
        pass
    else:
        raise AssertionError(f"Failed to raise {err_type} exception")
        
def assert_dt_equal(dt1, dt2):
    """
    datetime equality is a bit more complicated than it may seem when dealing
    with ambiguous and imaginary datetimes
    """
    
    fail_msg = f"{dt1} != {dt2}"
    assert dt1.astimezone(tz.UTC) == dt2.astimezone(tz.UTC), fail_msg
    assert dt1.tzname() == dt2.tzname(), fail_msg
    assert dt1.utcoffset() == dt2.utcoffset(), fail_msg
    assert dt1.dst() == dt2.dst(), fail_msg
        
NYC = tz.gettz('America/New_York')
with assert_raises(AmbiguousTimeError):
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=None)
    
with assert_raises(NonExistentTimeError):
    localize(datetime(2004, 4, 4, 2, 30), NYC, is_dst=None)

with assert_raises(ValueError):
    localize(datetime(2004, 10, 31, 1, 30, tzinfo=NYC), NYC)
    
assert_dt_equal(
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=False),
    datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=NYC))

assert_dt_equal(
    localize(datetime(2004, 10, 31, 1, 30), NYC, is_dst=True),
    datetime(2004, 10, 31, 1, 30, fold=0, tzinfo=NYC))

print("Passed!")

Passed!
